In [1]:
import pandas as pd 
import numpy as np 
import torch
from torch import nn
from datetime import datetime
import itertools
from classifier import *

mode = "train" | "predict"

In [2]:
learning_rate = 1e-6
num_iterations = 2000
mode = "test"
save_model = False
load_model = True
model_save_path = "./model" #it will have automatically appended the current date
model_load_path = "model2024-02-08_12_45.pt"

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
device

'cuda'

In [5]:
# this is the initialization line in the original code:
# models[m] = getattr(model_list, args.models[m].model)()
"""class Classifier(nn.Module):
    def __init__(self, dim_input, num_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(dim_input, 1500),
            nn.ReLU(),
            nn.Linear(1500, 780),
            nn.ReLU(),
            nn.Linear(780, num_classes),
        )
        self.softmax = nn.Softmax()

    # should return logits and features
    def forward(self, x):
        logits = self.model(x)
        prob = self.softmax(logits)
        return logits, prob"""

'class Classifier(nn.Module):\n    def __init__(self, dim_input, num_classes):\n        super().__init__()\n        self.model = nn.Sequential(\n            nn.Linear(dim_input, 1500),\n            nn.ReLU(),\n            nn.Linear(1500, 780),\n            nn.ReLU(),\n            nn.Linear(780, num_classes),\n        )\n        self.softmax = nn.Softmax()\n\n    # should return logits and features\n    def forward(self, x):\n        logits = self.model(x)\n        prob = self.softmax(logits)\n        return logits, prob'

In [6]:
df = pd.read_pickle("./comet_samsum_test_z_entire.pkl")

In [7]:
ohe_dict= \
{'xNeed': np.array([1,0,0,0,0], dtype=np.float32),
 'xIntent': np.array([0,1,0,0,0], dtype=np.float32),
 'HinderedBy': np.array([0,0,1,0,0], dtype=np.float32),
 'xWant':  np.array([0,0,0,1,0], dtype=np.float32),
 'xReason':  np.array([0,0,0,0,1], dtype=np.float32)}

In [8]:
df["cs_type_ohe"] = df["cs_type"].map(ohe_dict)
preprocessed = df[["sample_id","sentence_id", "cs_type_ohe","cs_encoded", "cos_similary_cs_summmary"]]

In [9]:
set(preprocessed["cs_encoded"].transform(len))

{384}

In [10]:
preprocessed["complete_target_X"] =preprocessed[["cs_type_ohe","cs_encoded"]].apply(lambda row: np.append(row[0],row[1]), axis=1)

C:\Users\NonAv\AppData\Local\Temp\ipykernel_15756\3403830363.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  preprocessed["complete_target_X"] =preprocessed[["cs_type_ohe","cs_encoded"]].apply(lambda row: np.append(row[0],row[1]), axis=1)
C:\Users\NonAv\AppData\Local\Temp\ipykernel_15756\3403830363.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed["complete_target_X"] =preprocessed[["cs_type_ohe","cs_encoded"]].apply(lambda row: np.append(row[0],row[1]), axis=1)


In [11]:
new_df = preprocessed.groupby(['sample_id','sentence_id']).agg(X=("complete_target_X",lambda x: list(itertools.chain.from_iterable(x))),Y=('cos_similary_cs_summmary',list))

new_df.reset_index() 

,sample_id,sentence_id,X,Y
0,13611370,0,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0059203124, -0.009...","[0.3164701759815216, 0.3950572609901428, 0.384..."
1,13611370,1,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.041891556, 0.07421...","[0.40869197249412537, 0.4197171926498413, 0.43..."
2,13611370,2,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.06959672, 0.007380...","[0.31288942694664, 0.3575688898563385, 0.34350..."
3,13611370,3,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.020754037, -0.0092...","[0.07747261971235275, 0.0885782241821289, 0.08..."
4,13611370,4,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.022999126, -0.0048...","[0.08124927431344986, 0.07299420237541199, 0.0..."
...,...,...,...,...
11834,13865481,11,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.15835816, 0.047548...","[0.12177974730730057, 0.16737879812717438, 0.0..."
11835,13865481,12,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.07220793, -0.02518...","[0.06631159037351608, 0.17811435461044312, 0.1..."
11836,13865481,13,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.044694398, -0.0544...","[0.24486178159713745, 0.2791087031364441, 0.20..."
11837,13865481,14,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.09239984, -0.01345...","[0.1931067705154419, 0.22074195742607117, 0.20..."


In [12]:
model = Classifier(dim_input=9725, num_classes=25).to(device)
if load_model:
    model.load_state_dict(torch.load(model_load_path))

In [13]:
sm =  nn.Softmax(dim=1)

In [14]:
X = torch.tensor(new_df["X"]).to(device)
Y = sm(torch.tensor(new_df["Y"])).to(device)

C:\Users\NonAv\AppData\Local\Temp\ipykernel_15756\1831164665.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  X = torch.tensor(new_df["X"]).to(device)
C:\Users\NonAv\AppData\Local\Temp\ipykernel_15756\1831164665.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Y = sm(torch.tensor(new_df["Y"])).to(device)


In [15]:
if mode == "train":
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    for t in range(num_iterations):
        # Forward pass: compute predicted y using operations on Tensors.
        optimizer.zero_grad()
        y_pred = model(X)

        # Compute and print loss using operations on Tensors.
        # Now loss is a Tensor of shape (1,)
        # loss.item() gets the scalar value held in the loss.
        loss = (y_pred - Y).pow(2).sum()
        if t % 100 == 99:
            print(t, loss.item())

        # Use autograd to compute the backward pass. This call will compute the
        # gradient of loss with respect to all Tensors with requires_grad=True.
        # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
        # the gradient of the loss with respect to a, b, c, d respectively.
        loss.backward()
        optimizer.step()

In [16]:
if save_model:
    torch.save(model.state_dict(), f"{model_save_path}{datetime.today().strftime('%Y-%m-%d_%H_%M')}.pt")

In [22]:
def test(X,Y, model, loss_fn):
    test_loss, correct = 0, 0
    with torch.no_grad():
        X, Y = X.to(device), Y.to(device)
        pred = model(X)
        test_loss = loss_fn(pred, Y)
        #correct = 
        for i,similarities in enumerate(pred):
            correct +=int(similarities.argmax() == Y[i].argmax())
    print(f"Test Error: \n Accuracy: {(100*correct/len(Y)):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [26]:
if mode == "test" or mode == "train":
    test(X,Y, model, lambda pred_logits, Y_test: (((pred_logits - Y_test).pow(2).sum(dim=-1)) * 1 / len(Y_test)).sum())

Test Error: 
 Accuracy: 23.1%, Avg loss: 0.886236 

